## Deliverable 2. Create a Customer Travel Destinations Map.

In [21]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Esperance,AU,-33.8667,121.9000,51.64,73,78,11.12,overcast clouds
1,Dikson,RU,73.5069,80.5464,8.42,92,90,20.85,overcast clouds
2,New Norfolk,AU,-42.7826,147.0587,58.50,86,3,2.80,overcast clouds
3,East London,ZA,-33.0153,27.9116,66.00,50,0,6.13,overcast clouds
4,San Pedro,PH,14.3500,121.0167,83.43,76,20,3.44,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,San Pedro,PH,14.3500,121.0167,83.43,76,20,3.44,overcast clouds
5,Belmonte,BR,-15.8631,-38.8828,76.46,85,4,8.08,overcast clouds
6,Avarua,CK,-21.2078,-159.7750,80.65,83,40,9.22,overcast clouds
11,Saint-Philippe,RE,-21.3585,55.7679,74.79,81,27,8.03,overcast clouds
18,Butaritari,KI,3.0707,172.7902,81.90,79,98,16.11,overcast clouds
22,Madimba,TZ,-10.4000,40.3333,75.92,87,96,11.83,overcast clouds
23,Rikitea,PF,-23.1203,-134.9692,77.47,66,31,21.36,overcast clouds
27,Kapaa,US,22.0752,-159.3190,78.78,83,100,11.50,overcast clouds
29,Cayenne,GF,4.9333,-52.3333,84.24,1,75,12.66,overcast clouds
33,Bengkulu,ID,-3.8004,102.2655,77.09,87,100,4.52,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   260
Country                258
Lat                    260
Lng                    260
Max Temp               260
Humidity               260
Cloudiness             260
Wind Speed             260
Current Description    260
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,San Pedro,PH,14.3500,121.0167,83.43,76,20,3.44,overcast clouds
5,Belmonte,BR,-15.8631,-38.8828,76.46,85,4,8.08,overcast clouds
6,Avarua,CK,-21.2078,-159.7750,80.65,83,40,9.22,overcast clouds
11,Saint-Philippe,RE,-21.3585,55.7679,74.79,81,27,8.03,overcast clouds
18,Butaritari,KI,3.0707,172.7902,81.90,79,98,16.11,overcast clouds
...,...,...,...,...,...,...,...,...,...
652,Mangulile,HN,15.0667,-86.8000,79.29,81,69,2.59,overcast clouds
654,Santiago,CL,-33.4569,-70.6483,70.99,45,0,12.66,overcast clouds
657,Wajid,SO,3.8093,43.2471,75.94,81,95,9.46,overcast clouds
658,Panguna,PG,-6.3164,155.4848,76.39,73,96,2.26,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,San Pedro,PH,83.43,overcast clouds,14.3500,121.0167,
5,Belmonte,BR,76.46,overcast clouds,-15.8631,-38.8828,
6,Avarua,CK,80.65,overcast clouds,-21.2078,-159.7750,
11,Saint-Philippe,RE,74.79,overcast clouds,-21.3585,55.7679,
18,Butaritari,KI,81.90,overcast clouds,3.0707,172.7902,
22,Madimba,TZ,75.92,overcast clouds,-10.4000,40.3333,
23,Rikitea,PF,77.47,overcast clouds,-23.1203,-134.9692,
27,Kapaa,US,78.78,overcast clouds,22.0752,-159.3190,
29,Cayenne,GF,84.24,overcast clouds,4.9333,-52.3333,
33,Bengkulu,ID,77.09,overcast clouds,-3.8004,102.2655,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,San Pedro,PH,83.43,overcast clouds,14.3500,121.0167,Esquivel's Garden And Resort
5,Belmonte,BR,76.46,overcast clouds,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
6,Avarua,CK,80.65,overcast clouds,-21.2078,-159.7750,Paradise Inn
11,Saint-Philippe,RE,74.79,overcast clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
18,Butaritari,KI,81.90,overcast clouds,3.0707,172.7902,Isles Sunset Lodge


In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df[hotel_df["Hotel Name"]!=""]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,San Pedro,PH,83.43,overcast clouds,14.3500,121.0167,Esquivel's Garden And Resort
5,Belmonte,BR,76.46,overcast clouds,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
6,Avarua,CK,80.65,overcast clouds,-21.2078,-159.7750,Paradise Inn
11,Saint-Philippe,RE,74.79,overcast clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
18,Butaritari,KI,81.90,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
...,...,...,...,...,...,...,...
644,Majene,ID,78.44,overcast clouds,-3.5403,118.9707,Kost Putri RABBANI
647,Tezu,IN,71.15,overcast clouds,27.9167,96.1667,Circuit House
654,Santiago,CL,70.99,overcast clouds,-33.4569,-70.6483,Hotel Panamericano
657,Wajid,SO,75.94,overcast clouds,3.8093,43.2471,xamdihotelresturant114


In [32]:
# 8a. Create the output File (CSV)
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")
                      

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))